	ber-language

	<program>		::= DISAS <var-decls> <statements> .leo

	<var-decls>		::= <var-decl> |
					<var-decl> <var-decls>

	<var-decl>		::= <type> REGISTER ;

				
	<type> 			::= long | int | hex | bin | ;

	<statements>		::= <open_statement> |
					<open_statement> <statements>

	<statement>		::= <atrib> |
					<if-else>

	<atrib>			::= REGISTER = <expression> ;

	<expression>		::= REGISTER |
					number |
					addw <expression>, <expression> |
					subw <expression>, <expression> |
					mul <expression>, <expression> |
					div <expression>, <expression> |
					( <expression> )





	<open_statement>	::= <atrib> |
				se-o-ber ( <expression> <comp> <expression> ){ <open_statement> } |
				se-o-ber ( <expression> <comp> <expression> ){ <closed_statement> } senao-o-ber { <open_statement> }



	<closed_statement>	::= <atrib> |
				se-o-ber ( <expression> <comp> <expression> ){ <closed_statement> } senao-o-ber { <closed_statement> }



					== |  != |   <   |   >  |     <=    |    >=
	<comp>			::= leo | ber | lucca | enzo | lucca-leo | enzo-leo





In [25]:
!pip install rply --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


**Implementação do Analisador Léxico**

In [26]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('BINUMBER', r'0b[01]+')
lg.add('HEXNUMBER', r'0x[0-9A-Fa-f]+')
lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')
lg.add('OPEN_BRACKET', r'\{')
lg.add('CLOSE_BRACKET', r'\}')

lg.add('INT', r'int')
lg.add('LONG', r'long')
lg.add('BIN', r'bin')
lg.add('HEX', r'hex')
lg.add('IF', r'se-o-ber')
lg.add('ELSE', r'senao-o-ber')
lg.add('WHILE', r'enquanto-o-ber')

lg.add('START', 'disas')
lg.add('END', '\.leo')
lg.add('COMP','leo')
lg.add('COMP','ber')
lg.add('COMP','enzo-leo')
lg.add('COMP','enzo')
lg.add('COMP','lucca-leo')
lg.add('COMP','lucca')
lg.add('REGISTER', r'[a-zA-Z][a-zA-Z0-9]*')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

In [27]:
program = """
disas
    bin REGISTER1;

    REGISTER1 = 0b101010;
.leo
"""



a = lexer.lex(program)

for i in a:
    print(i)

Token('START', 'disas')
Token('BIN', 'bin')
Token('REGISTER', 'REGISTER1')
Token('SEMICOL', ';')
Token('REGISTER', 'REGISTER1')
Token('EQUALS', '=')
Token('BINUMBER', '0b101010')
Token('SEMICOL', ';')
Token('END', '.leo')


In [28]:
program2 = """
disas
    int REGISTER1;
    bin REGISTER2;

    se-o-ber (REGISTER1 leo 10) {
        REGISTER1 = 20;
    } senao-o-ber {
        REGISTER2 = 0b101010;
    }

    enquanto-o-ber (REGISTER2 lucca 0b111111) {
        REGISTER2 = REGISTER2 + 0b1;
    }

    hex REGISTER3 = 0x1af;
    
.leo
"""

a = lexer.lex(program2)

for i in a:
    print(i)

Token('START', 'disas')
Token('INT', 'int')
Token('REGISTER', 'REGISTER1')
Token('SEMICOL', ';')
Token('BIN', 'bin')
Token('REGISTER', 'REGISTER2')
Token('SEMICOL', ';')
Token('IF', 'se-o-ber')
Token('OPEN_PARENS', '(')
Token('REGISTER', 'REGISTER1')
Token('COMP', 'leo')
Token('NUMBER', '10')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACKET', '{')
Token('REGISTER', 'REGISTER1')
Token('EQUALS', '=')
Token('NUMBER', '20')
Token('SEMICOL', ';')
Token('CLOSE_BRACKET', '}')
Token('ELSE', 'senao-o-ber')
Token('OPEN_BRACKET', '{')
Token('REGISTER', 'REGISTER2')
Token('EQUALS', '=')
Token('BINUMBER', '0b101010')
Token('SEMICOL', ';')
Token('CLOSE_BRACKET', '}')
Token('WHILE', 'enquanto-o-ber')
Token('OPEN_PARENS', '(')
Token('REGISTER', 'REGISTER2')
Token('COMP', 'lucca')
Token('BINUMBER', '0b111111')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACKET', '{')
Token('REGISTER', 'REGISTER2')
Token('EQUALS', '=')
Token('REGISTER', 'REGISTER2')
Token('PLUS', '+')
Token('BINUMBER', '0b1')
Token('SEMICOL',

In [29]:
from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self, cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class BinNumber(Expr):
    def __init__(self, value):
        self.value = value

class HexNumber(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


**Implementando o Analisador Sintático**

In [30]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['BINUMBER', 'HEXNUMBER', 'NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'OPEN_BRACKET', 'CLOSE_BRACKET', 'PLUS', 'MINUS', 'MUL', 'DIV',
     'INT', 'LONG', 'BIN', 'HEX', 'IF', 'ELSE', 'WHILE', 'START', 'END',
     'COMP', 'EQUALS', 'SEMICOL', 'REGISTER'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : START var-decls statements END')
def prog(p):
    return Prog(p[1],p[2])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('var-decls : var-decl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('var-decls : var-decl var-decls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('var-decl : LONG REGISTER SEMICOL')
def vardecl_long(p):
    return VarDecl(p[1].getstr(), "long")

@pg.production('var-decl : INT REGISTER SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int")

@pg.production('var-decl : HEX REGISTER SEMICOL')
def vardecl_hex(p):
    return VarDecl(p[1].getstr(), "hex")

@pg.production('var-decl : BIN REGISTER SEMICOL')
def vardecl_bin(p):
    return VarDecl(p[1].getstr(), "bin")

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : open_statement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : open_statement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('open_statement : REGISTER EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('open_statement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET open_statement CLOSE_BRACKET')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],None)


@pg.production('open_statement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET closed_statement CLOSE_BRACKET ELSE OPEN_BRACKET open_statement CLOSE_BRACKET')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],p[11])

@pg.production('open_statement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET open_statement CLOSE_BRACKET')
def statement_while(p):
    return While (p[2],p[3],p[4],p[7])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closed_statement : REGISTER EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closed_statement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET closed_statement CLOSE_BRACKET ELSE closed_statement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],p[10])

@pg.production('closed_statement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET closed_statement CLOSE_BRACKET')
def statement_while(p):
    return While (p[2],p[3],p[4],p[7])

@pg.production('expression : REGISTER')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : BINUMBER')
def expression_binumber(p):
    return BinNumber(p[0].getstr())

@pg.production('expression : HEXNUMBER')
def expression_hexnumber(p):
    return HexNumber(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

/tmp/ipykernel_18184/524115693.py:134: ParserGeneratorWarning: 1 reduce/reduce conflict
  parser = pg.build()
